In [1]:
import pandas as pd

In [2]:
d20_21 = pd.read_excel('DATOS HISTÓRICOS 2020_2021_TODAS ESTACIONES.xlsx', sheet_name='SURESTE3')
d22_23 = pd.read_excel('DATOS HISTÓRICOS 2022_2023_TODAS ESTACIONES.xlsx', sheet_name='SURESTE3')
df = pd.concat([d20_21,d22_23])

In [3]:
df['date'] = pd.to_datetime(df['date'])
df["deltaDate"] = (df["date"] - df.iloc[0,0]).apply(lambda x: x.value/3.6e+12)
df.head()

,date,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,deltaDate
0,2020-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,37.0,18.0,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.0
1,2020-01-01 01:00:00,1.88,2.0,5.2,7.2,25.0,37.0,28.0,734.2,0.01,93.0,NaN,0.037,11.68,1.7,98.0,1.0
2,2020-01-01 02:00:00,1.86,2.6,5.0,7.6,22.0,55.0,28.0,733.8,0.02,93.0,NaN,0.037,11.68,1.6,133.0,2.0
3,2020-01-01 03:00:00,1.78,3.1,5.9,8.9,21.0,35.0,11.0,733.0,0.00,93.0,NaN,0.037,11.70,1.5,198.0,3.0
4,2020-01-01 04:00:00,1.76,2.0,4.5,6.5,22.0,27.0,NaN,732.8,0.02,93.0,NaN,0.037,11.80,0.7,240.0,4.0


In [4]:
from sklearn.impute import KNNImputer as imputer
k = imputer(n_neighbors=4)
datos = k.fit_transform((df.iloc[:, 1:]).to_numpy())

In [5]:
limpios = pd.DataFrame(datos, columns=df.columns[1:])
limpios["date"] = list(df["date"])
limpios.to_csv("datosCompletos.csv")

In [17]:
intervalos={
    "SO2":[0.008, 0.11, 0.165, 0.22],
    "NO2":[8.75, 11, 13.3, 15.5],
    "etiqueta": ["bueno", "aceptable", "mala", "muy mala"]
}
def change(val, col):
    if val< intervalos[col][0]:
        return "bueno"
    if val< intervalos[col][1]:
        return "aceptable"
    if val< intervalos[col][2]:
        return "mala"
    if val< intervalos[col][3]:
        return "muy mala"
    return "Extremadamente mala"

In [18]:
limpios["NO2_discret"] = limpios["NO2"].apply(lambda x: change(x, "NO2"))
limpios["SO2_discret"] = (limpios["SO2"]*0.001).apply(lambda x: change(x, "SO2"))

In [19]:
limpios["SO2_pm"] = limpios["SO2"]*0.001

In [20]:
limpios.head()

,CO,NO,NO2,NOX,O3,PM10,PM2.5,PRS,RAINF,RH,SO2,SR,TOUT,WSR,WDR,deltaDate,date,NO2_discret,SO2_discret,SO2_pm
0,1.8775,4.3,6.375,10.625,19.25,37.0,18.00,733.025,0.015,92.75,0.850,0.000,12.2675,1.325,207.25,0.0,2020-01-01 00:00:00,bueno,bueno,0.000850
1,1.8800,2.0,5.200,7.200,25.00,37.0,28.00,734.200,0.010,93.00,1.775,0.037,11.6800,1.700,98.00,1.0,2020-01-01 01:00:00,bueno,bueno,0.001775
2,1.8600,2.6,5.000,7.600,22.00,55.0,28.00,733.800,0.020,93.00,0.750,0.037,11.6800,1.600,133.00,2.0,2020-01-01 02:00:00,bueno,bueno,0.000750
3,1.7800,3.1,5.900,8.900,21.00,35.0,11.00,733.000,0.000,93.00,0.725,0.037,11.7000,1.500,198.00,3.0,2020-01-01 03:00:00,bueno,bueno,0.000725
4,1.7600,2.0,4.500,6.500,22.00,27.0,10.25,732.800,0.020,93.00,0.775,0.037,11.8000,0.700,240.00,4.0,2020-01-01 04:00:00,bueno,bueno,0.000775


In [8]:
datos = limpios[["date", "deltaDate","NO","NO2", "NOX", "SO2", "TOUT", "RH", "SR"]]
datos.to_csv("datos.csv")

In [21]:
datos = limpios[["date", "deltaDate","NO","NO2", "NOX", "SO2", "TOUT", "RH", "SR", "NO2_discret",	"SO2_discret",	"SO2_pm"]]

In [22]:
datos.head()

,date,deltaDate,NO,NO2,NOX,SO2,TOUT,RH,SR,NO2_discret,SO2_discret,SO2_pm
0,2020-01-01 00:00:00,0.0,4.3,6.375,10.625,0.850,12.2675,92.75,0.000,bueno,bueno,0.000850
1,2020-01-01 01:00:00,1.0,2.0,5.200,7.200,1.775,11.6800,93.00,0.037,bueno,bueno,0.001775
2,2020-01-01 02:00:00,2.0,2.6,5.000,7.600,0.750,11.6800,93.00,0.037,bueno,bueno,0.000750
3,2020-01-01 03:00:00,3.0,3.1,5.900,8.900,0.725,11.7000,93.00,0.037,bueno,bueno,0.000725
4,2020-01-01 04:00:00,4.0,2.0,4.500,6.500,0.775,11.8000,93.00,0.037,bueno,bueno,0.000775


In [23]:
datos.to_csv("limpios.csv")